In [ ]:
import pandas as pd
import re
import math
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import sentencepiece as spm

from tqdm import tqdm
import random

file = "amazon_review.csv"

data = pd.read_csv(file).dropna(ignore_index=True)
data['overall'] = data["overall"] - 1

vocab_size = 16000
seq_len = 512
pad_token = 16008

spm.SentencePieceTrainer.train(
    input='amazon_reviews.txt',
    model_prefix='amazon_reviews',
    vocab_size=vocab_size,
    model_type='unigram',
    character_coverage=1.0
)

tok = spm.SentencePieceProcessor(model_file='amazon_reviews.model')

filter_ = 1

ls  = [len(tok.encode(i, out_type=int)) for i in data["reviewText"]]

data["lengths"] = ls

data_trunc = data[data["lengths"]<=80]

v = data_trunc["overall"].value_counts()

balanced_data = (
    data_trunc.groupby("overall")
      .sample(n=min(v), random_state=42)
      .reset_index(drop=True)
)

len(balanced_data)

df_shuffled = balanced_data.sample(frac=filter_, random_state=42).reset_index(drop=True)

# Split the shuffled DataFrame
train_size = 0.8
train_df = df_shuffled.sample(frac=train_size, random_state=42).reset_index(drop=True)
test_df = df_shuffled.drop(train_df.index).reset_index(drop=True)

print("len train df: ", len(train_df))
print("len test df: ", len(test_df))

# get one input output pair with special tokens for later concatenation with other pairs for one sequence under max sequence length
def getEncodingOpen(df, i):
    reviewtext = "Review: "+ df["reviewText"].iloc[i]
    rating = "Rating: " 
    #row = [16000] + tok.encode(reviewText, out_type=int) + [16001] + tok.encode([int(df["overall"].iloc[i])], out_type = int)
    row = [16000] + tok.encode(reviewtext, out_type = int) + [16002] + tok.encode(rating, out_type = int)
    correct_output_rating = int(df["overall"].iloc[i])
    row = torch.LongTensor(row)
    correct_output_rating = torch.LongTensor([correct_output_rating])
    return row, correct_output_rating
# get one input output pair with special tokens for later concatenation with other pairs for one sequence under max sequence length
def getEncoding(df, i):
    reviewtext = "Review: "+ df["reviewText"].iloc[i]
    rating = "Rating: " #+ str(int(df["overall"].iloc[i]))
    score = int(df["overall"].iloc[i])
    row = [16000] + tok.encode(reviewtext, out_type = int) + [16002] + tok.encode(rating, out_type = int) +[score+16003]+ [16001]
    return row


d = {16000: "<BOS>", 16001: "<EOS>",16002: "<SEP>",16003: "<0>",16004: "<1>",16005: "<2>",16006: "<3>",16007: "<4>", 16008: "PAD"}

def decode_seq(seq):
    outp = ""
    sofar = []
    for i in seq:
        if(i<=15999):
            sofar.append(i)
        else:
            outp += tok.decode(sofar)
            outp += d[i]
            sofar = []
    outp += tok.decode(sofar)
    return outp


decode_seq(getEncoding(train_df, 2))

train_df["overall"].iloc[2]

def getShiftSeq(df_t, max_seq=1024):
    seqs_x = []
    seqs_y = []
    seqs = []
    c = []
    for i in range(len(df_t)):
        row = getEncoding(df_t, i)
        if len(c) + len(row) > max_seq +1:
            seqs_x.append(c[:-1])
            seqs_y.append(c[1:])
            seqs.append(c)
            c = []
        c.extend(row)
    return seqs_x, seqs_y, seqs

train_seqs_x, train_seqs_y, train_seqs = getShiftSeq(train_df, max_seq=seq_len)
test_seqs_x, test_seqs_y, test_seqs = getShiftSeq(test_df, max_seq=seq_len)

len(train_seqs_x[0])

len(train_seqs[0])

decode_seq(train_seqs[0][-15:])

train_seqs_x[0][-15:]

train_seqs[0][-15:]

class TokenDatasetB(Dataset):
    def __init__(self, seqs_x, seqs_y):
        self.seqs_x = seqs_x
        self.seqs_y = seqs_y
    
    def __len__(self):
        return len(self.seqs_x)
    
    def __getitem__(self, idx):
        return torch.LongTensor(self.seqs_x[idx]), torch.LongTensor(self.seqs_y[idx])

# Padding collate function for variable length sequences
def collate_fnB(batch):
    seqs_x, seqs_y = zip(*batch)
    lens = [len(s) for s in seqs_x]
    max_len = max(lens)
    
    # Pad sequences
    padded_x = torch.zeros(len(seqs_x), max_len, dtype=torch.long) + pad_token
    padded_y = torch.zeros(len(seqs_y), max_len, dtype=torch.long) + pad_token
    
    for i, (x, y) in enumerate(zip(seqs_x, seqs_y)):
        padded_x[i, :len(x)] = x
        padded_y[i, :len(y)] = y
    
    return padded_x, padded_y, torch.LongTensor(lens)

dataset = TokenDatasetB(train_seqs_x, train_seqs_y)
train_loader = DataLoader(dataset, batch_size=5, shuffle=True, collate_fn=collate_fnB)

for batch_idx, (x, y, lengths) in enumerate(train_loader):
    print("bi")
    print(batch_idx)
    print("x")
    print(x.shape)
    xl = x[0].tolist()
    print(xl)
    print()
    
    
    print(decode_seq(x[0].tolist()))
    print()
    print(decode_seq(x[1].tolist()))
    print()
    print(decode_seq(x[2].tolist()))
    print()
    print(decode_seq(x[3].tolist()))
    print()
    print(decode_seq(x[4].tolist()))
    print()
    print("y")
    print(y)
    print(y.shape)
    print("lengths")
    print(lengths.shape)
    break

if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device name: {torch.cuda.get_device_name(0)}")
    device = 'cuda'
else:
    print("CUDA is not available. PyTorch will use the CPU.")
    device = 'cpu'

class GPTBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super().__init__()

        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(
            embed_dim, num_heads, dropout=dropout, batch_first=True
        )

        self.ln2 = nn.LayerNorm(embed_dim)

        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x, causal_mask, padding_mask):
        # Self-attention (GPT-style)
        h = self.ln1(x)
        attn_out, _ = self.attn(
            h, h, h,
            attn_mask=causal_mask,
            key_padding_mask=padding_mask,
            need_weights=False
        )
        x = x + attn_out

        # Feedforward
        h = self.ln2(x)
        ff_out = self.mlp(h)
        x = x + ff_out

        return x
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, max_len=512,
                 embed_dim=1024, num_heads=4,
                 num_layers=4, mlp_dim=2048, dropout=0.2):
        super().__init__()

        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos = nn.Embedding(max_len, embed_dim)

        self.layers = nn.ModuleList([
            GPTBlock(embed_dim, num_heads, mlp_dim, dropout)
            for _ in range(num_layers)
        ])

        self.ln_final = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, vocab_size, bias=False)
        #self.head.weight = self.embed.weight  # weight tying

    #def causal_mask(self, T, device):
    #    mask = torch.triu(torch.ones(T, T, device=device), 1)
    #    return mask * float("-inf")

    def causal_mask(self, T, device): 
        mask = torch.triu(torch.ones(T, T, device=device), 1) 
        return mask.masked_fill(mask == 1, float('-inf'))



    
    def forward(self, x, lengths=None):
        B, T = x.shape
        device = x.device

        tok = self.embed(x)
        pos = self.pos(torch.arange(T, device=device)[None, :])
        h = tok + pos

        causal = self.causal_mask(T, device)     # (T, T)
        pad_mask = (x == 0)                      # (B, T)

        for layer in self.layers:
            h = layer(h, causal, pad_mask)

        h = self.ln_final(h)
        return self.head(h)                      # (B, T, V)


# Training function
def train_model(model, train_loader, epochs=10, lr=1e-4, device='cuda'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    track_loss = []
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loader = tqdm(train_loader)
        
        for x, y, lengths in loader:
            x, y = x.to(device), y.to(device)
            lengths = lengths.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            logits = model(x, lengths)
            
            # Reshape for loss calculation
            logits = logits.view(-1, logits.size(-1))
            y = y.view(-1)
            
            loss = criterion(logits, y)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            track_loss.append(loss.item())
            avg_loss = sum(track_loss[-10:]) / 10
            loader.set_postfix(loss=avg_loss)
            del logits
            torch.cuda.empty_cache()
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}')
    
    return model




dataset = TokenDatasetB(train_seqs_x, train_seqs_y)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fnB)


# Initialize model (set vocab_size to your tokenizer's vocab size + special tokens)
vocab_size = 16000+9  # Adjust based on your tokenizer
model = DecoderOnlyTransformer(vocab_size=vocab_size,  num_layers=12)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

# Train
model = train_model(model, train_loader, epochs=20, lr=1e-4)


k = [345, 23, 44, 99]
kt = torch.tensor(k)

seq_len

anno, score = getEncodingOpen(test_df, random.randint(0,len(test_df)))

annot = torch.tensor(anno)

decode_seq(anno.tolist())

annot

score

lens = len(anno)
anno_len = torch.tensor([lens])

# Pad sequences
padded_x = torch.zeros(1, 512, dtype=torch.long) + pad_token
padded_x[0,:lens] = torch.tensor(anno)

padded_x = padded_x.to(device)
anno_len = anno_len.to(device)

padded_x[0].shape

decode_seq(padded_x[0].tolist())

out = model(padded_x, anno_len)

out.shape

tokenout = torch.argmax(out,dim=2)[0]

score

tokenout.shape

decode_seq(tokenout.tolist())



class ICLDataset(Dataset):
    def __init__(self, df, shots, seq_len):
        self.df = df
        self.shots = shots
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.df)

    def getpre(self, idx, l):
        for _ in range(10):
            ixs = []
            xs = []
            for j in range(self.shots):
                sel = idx
                while(sel==idx):
                    sel = random.randint(0, self.__len__() -1)
                xs.extend(getEncoding(self.df, sel))
            if(len(xs) + l <= seq_len):
                return torch.LongTensor(xs)
        raise ValueError(f"can't fit {self.shots} examples in context")
                
            
    
    def __getitem__(self, idx):
        x, y = getEncodingOpen(self.df, idx)
        l = len(x)
        pre = self.getpre(idx, l)
        icl_x = torch.cat((pre,x))
        return icl_x, y

# Padding collate function for variable length sequences
def collate_fn_icl(batch):
    seqs_x, y = zip(*batch)
    lens = [len(s) for s in seqs_x]
    max_len = max(lens)
    
    # Pad sequences
    padded_x = torch.zeros(len(seqs_x), max_len, dtype=torch.long) + pad_token
    
    for i, x in enumerate(seqs_x):
        padded_x[i, :len(x)] = x
    
    return padded_x, torch.LongTensor(y), torch.LongTensor(lens)



icl_dataset = ICLDataset(train_df, 0, seq_len)
icl_loader = DataLoader(icl_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn_icl)

for batch_idx, (x, y, lengths) in enumerate(icl_loader):
    print("bi")
    print(batch_idx)
    print("x")
    print(x.shape)
    xl = x[0].tolist()
    print(xl)
    print()
    
    
    print(decode_seq(xl))
    print("y")
    print(y)
    print(y.shape)
    print("lengths")
    print(lengths.shape)
    break

def israting(s):
    bnk = ["0","1","2","3","4"]
    return s in bnk
def isnum(s):
    bnk = [str(n) for n in range(0,20)]
    return s in bnk

def check_token_list(token_list):
    isratings = 0
    isnums = 0
    for token in token_list:
        isratings += 1 if israting(token) else 0
        isnums += 1 if isnum(token) else 0
    return isratings, isnums

tok.decode([15999])

tok.decode([])

pred

d

def get_score(x):
    x = x.tolist()
    for i in range(len(x)-1,0, -1):
        if 16003<= x[i] <=16007:
            return x[i]
    return 20000


test_ex = torch.tensor([ 7421,   292,     4,     6,   945,    35,   161,    19,    31,   161,
             6,    18,   161,    19,    10,    29,    26,   143,   204,     3,
            28,   321,    14,   143,    75,   460, 13254,   292, 16000, 16008,
         16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008,
         16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008,
         16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008, 16008,
         16008])
vv = get_score(test_ex)

vv

def getTokenScore(t):
    if(t==20000):
        return '-'
    return d[t][1]

icl_dataset = ICLDataset(train_df, 0, seq_len)
icl_loader = DataLoader(icl_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn_icl)

tot = 0
num = 0
ratings = 0
correct = 0
for batch_idx, (x, y, lengths) in enumerate(icl_loader):
    x = x.to(device)
    y = y.to(device)
    lengths = lengths.to(device)
    pred_logits = model(x,lengths)
    print(x.shape)
    print(x[0])
    for sequence in x:
        print(decode_seq(sequence.tolist()))
        print()
    print("========================")
    pred_tokens = torch.argmax(pred_logits,dim=2)
    pred_tokens = pred_tokens.to('cpu')
    #pred_tokens = trim_tail(pred_tokens, pad_token)
    #pred_last_token = pred_tokens[:,-1].tolist()
    pred_last_token = [get_score(i) for i in pred_tokens]
    for sequence in pred_tokens:
        print(decode_seq(sequence.tolist()))
        print()
    pred_scores = []
    print(pred_last_token)
    for token in pred_last_token:
        if(token<=15999):
            pred_scores.append(tok.decode([token]))
        else:
            pred_scores.append(getTokenScore(token))

    print('---')
    print(pred_scores)
    
    ys = [str(ans) for ans in y.tolist()]
    print(ys)
    tot += len(y)
    for t in range(len(y)):
        correct += 1 if pred_scores[t] == ys[t] else 0
    isratings, isnums = check_token_list(pred_scores)
    num += isnums
    ratings += isratings
    torch.cuda.empty_cache()
    break

print("tot: ", tot)
print("num: ", num)
print("ratings: ", ratings)
print("correct: ", correct)

icl_dataset = ICLDataset(train_df, 1, seq_len)
icl_loader = DataLoader(icl_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn_icl)

tot = 0
num = 0
ratings = 0
correct = 0
for batch_idx, (x, y, lengths) in enumerate(icl_loader):
    x = x.to(device)
    y = y.to(device)
    lengths = lengths.to(device)
    pred_logits = model(x,lengths)
    #print(x.shape)
    #print(x[0])
    #for sequence in x:
    #    print(decode_seq(sequence.tolist()))
    #    print()
    #print("========================")
    pred_tokens = torch.argmax(pred_logits,dim=2)
    pred_tokens = pred_tokens.to('cpu')
    #pred_tokens = trim_tail(pred_tokens, pad_token)
    #pred_last_token = pred_tokens[:,-1].tolist()
    pred_last_token = [get_score(i) for i in pred_tokens]
    #for sequence in pred_tokens:
    #    print(decode_seq(sequence.tolist()))
    #    print()
    pred_scores = []
    #print(pred_last_token)
    for token in pred_last_token:
        if(token<=15999):
            pred_scores.append(tok.decode([token]))
        else:
            pred_scores.append(getTokenScore(token))

    #print('---')
    #print(pred_scores)
    
    ys = [str(ans) for ans in y.tolist()]
    #print(ys)
    tot += len(y)
    for t in range(len(y)):
        correct += 1 if pred_scores[t] == ys[t] else 0
    isratings, isnums = check_token_list(pred_scores)
    num += isnums
    ratings += isratings
    torch.cuda.empty_cache()

print("tot: ", tot)
print("num: ", num)
print("ratings: ", ratings)
print("correct: ", correct)
print("acc: ", correct/num)

5358/15387 #0

 4778 / 15396

def run_icl_experiment(model, test_df, tokenizer, shot_counts=[0, 1, 3, 5, 10], num_samples=500, device='cuda'):
    model.eval()
    results = {}
    
    print(f"Running ICL Experiment on {num_samples} samples per shot count...")
    
    # Pre-compute samples to ensure fairness (same targets for all shots?)
    # For simplicity and to match the previous logic, we'll generate fresh samples for each shot count
    # or we can try to keep targets consistent. Let's follow the notebook's pattern.
    
    for k in shot_counts:
        print(f"\nEvaluating {k}-shot performance...")
        correct = 0
        total = 0
        
        # We need to construct the prompts manually since ICLDataset logic was in a different notebook
        # and might rely on specific variables. Let's implement a self-contained loop.
        
        for i in range(num_samples):
            # 1. Pick a target
            target_idx = random.randint(0, len(test_df)-1)
            target_row = test_df.iloc[target_idx]
            target_review = str(target_row['reviewText'])
            target_label = int(target_row['overall'])
            
            # 2. Pick k shots (excluding target)
            shots_df = test_df.drop(target_idx)
            if k > 0:
                shots = shots_df.sample(n=k)
            else:
                shots = pd.DataFrame()
            
            # 3. Construct Prompt
            # Format: [Review 1] [Rating 1] ... [Review Target] [Rating: ]
            prompt_ids = [16000] # Start token if needed, or just start concatenation
            # Actually, let's stick to the format used in training/prediction
            # The model expects: [16000] Review [16002] Rating: [Label] [16001]
            
            full_prompt_ids = []
            
            # Add shots
            for _, shot_row in shots.iterrows():
                shot_review = str(shot_row['reviewText'])
                shot_label = int(shot_row['overall'])
                
                # Shot encoding: [16000] Review [16002] Rating: Label [16001]
                # Note: We must ensure this matches exactly what the model learned.
                # In getEncoding: [16000] + review + [16002] + "Rating: " + label + [16001]
                
                row_ids = [16000] + tokenizer.encode(shot_review, out_type=int) + \
                          [16002] + tokenizer.encode("Rating: ", out_type=int) + \
                          [shot_label + 16003] + [16001]
                full_prompt_ids.extend(row_ids)
            
            # Add target (without label)
            # Target encoding: [16000] Review [16002] Rating:
            target_ids = [16000] + tokenizer.encode(target_review, out_type=int) + \
                         [16002] + tokenizer.encode("Rating: ", out_type=int)
            
            full_prompt_ids.extend(target_ids)
            
            # 4. Truncate if too long (simple truncation from left)
            # Max seq len is 512. If prompt is longer, we lose the start.
            # Ideally we keep the target and as many shots as possible.
            if len(full_prompt_ids) > 1024: # giving some buffer, though model trained on 1024
                 full_prompt_ids = full_prompt_ids[-1024:]
            
            # 5. Predict
            x = torch.LongTensor([full_prompt_ids]).to(device)
            
            with torch.no_grad():
                logits = model(x)
                last_logits = logits[0, -1, 🙂
                pred_id = torch.argmax(last_logits).item()
            
            pred_rating = pred_id - 16003
            
            if pred_rating == target_label:
                correct += 1
            total += 1
        
        acc = correct / total
        results[k] = acc
        print(f"{k}-shot Accuracy: {acc:.4f}")
        
    return results

# Run the experiment
if 'model' in locals():
    icl_results = run_icl_experiment(model, test_df, tok, shot_counts=[0, 1, 3, 5, 8, 10], num_samples=200)
    print("\nFinal Results:", icl_results)
else:
    print("Model not loaded.")